In [21]:
import os
import numpy as np
import csv
import matplotlib.pyplot as plt

from tslearn.utils import to_time_series_dataset
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

In [31]:
# helper functions

def series_append(series, list, keys):
    for i in range(64):
        series[keys[i]].append(float(list[i]))
    return series

def load_series(filename):
    with open(filename, 'r') as csv_in:
        csv_file = list(csv.reader(csv_in))
        series = {}
        keys = csv_file[0]
        for key in keys: series[key] = []
        for i in range(2, len(csv_file), 2):
            series = series_append(series, csv_file[i], keys)
        return [series, int((len(csv_file) - 2) / 2)]

def series_to_arr(series):
    l = []
    for key in series:
        if (key[-1] == 'v'): continue
        l += (series[key])
    return l

def E3(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 3):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E2(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 2):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E1(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 1):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E0(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) == 0):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

In [32]:
# loading train and test sets

raw_X_train = []
raw_X_test = []
smoothed_X_train = []
smoothed_X_test = []
translate_X_train = []
translate_X_test = []
final_X_train = []
final_X_test = []

o_label_train = []
o_label_test = []

f_label_train = []
f_label_test = []

b_label_train = []
b_label_test = []

s_label_train = []
s_label_test = []

l_label_train = []
l_label_test = []

with open("..\\training_examples.txt") as t:
    training_examples = t.readlines()

with open("..\\test_examples.txt") as t:
    test_examples = t.readlines()

for file in training_examples:
    raw_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\1_unprocessed\\' + file[:-1])
    smoothed_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\2_smoothed\\' + file[:-1])
    translate_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\3_translation\\' + file[:-1])
    final_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\4_final\\' + file[:-1])

    raw_X_train.append(series_to_arr(raw_series))
    smoothed_X_train.append(series_to_arr(smoothed_series))
    translate_X_train.append(series_to_arr(translate_series))
    final_X_train.append(series_to_arr(final_series))

    o_label_train.append(int(file[4]))
    f_label_train.append(int(file[7]))
    b_label_train.append(int(file[10]))
    s_label_train.append(int(file[13]))
    l_label_train.append(int(file[16]))

for file in test_examples:
    raw_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\1_unprocessed\\' + file[:-1])
    smoothed_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\2_smoothed\\' + file[:-1])
    translate_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\3_translation\\' + file[:-1])
    final_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\4_final\\' + file[:-1])

    raw_X_test.append(series_to_arr(raw_series))
    smoothed_X_test.append(series_to_arr(smoothed_series))
    translate_X_test.append(series_to_arr(translate_series))
    final_X_test.append(series_to_arr(final_series))

    o_label_test.append(int(file[4]))
    f_label_test.append(int(file[7]))
    b_label_test.append(int(file[10]))
    s_label_test.append(int(file[13]))
    l_label_test.append(int(file[16]))

raw_X_train = to_time_series_dataset(raw_X_train)
raw_X_test = to_time_series_dataset(raw_X_test)

smoothed_X_train = to_time_series_dataset(smoothed_X_train)
smoothed_X_test = to_time_series_dataset(smoothed_X_test)

translate_X_train = to_time_series_dataset(translate_X_train)
translate_X_test = to_time_series_dataset(translate_X_test)

final_X_train = to_time_series_dataset(final_X_train)
final_X_test = to_time_series_dataset(final_X_test)

print(l_label_train)

[1, 1, 7, 7, 8, 8, 2, 2, 2, 2, 7, 7, 8, 8, 7, 7, 8, 9, 3, 3, 3, 3, 7, 7, 2, 2, 1, 5, 7, 7, 4, 9, 9, 5, 5, 2, 7, 6, 6, 4, 3, 7, 7, 6, 6, 4, 2, 2, 6, 6, 6, 6, 7, 2, 6, 6, 3, 3, 2, 2, 9, 4, 4, 2, 9, 9, 1, 1, 5, 5, 4, 4, 7, 5, 5, 7, 7, 5, 5, 6, 6, 6, 6, 6, 8, 9, 9, 2, 2, 4, 4, 7, 7, 0, 0, 0, 0, 0, 1, 0, 0, 7, 7, 7, 7, 5, 5, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 7, 7, 8, 8, 0, 0, 0, 1, 1, 3, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 3, 3, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 5, 5, 0, 0, 3, 3, 5, 4, 0, 3, 3, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 3, 1, 3, 3, 1, 0, 0, 1, 6, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 3, 3, 2, 2, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 7, 7, 1, 1, 3, 3, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 1, 2, 2, 0, 0, 0, 0, 1, 1, 6, 0, 4, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 6, 0, 

In [33]:
# checking accuracies for KNN
oknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
oknn.fit(raw_X_train, o_label_train)
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
fknn.fit(raw_X_train, f_label_train)
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
bknn.fit(raw_X_train, b_label_train)
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
sknn.fit(raw_X_train, s_label_train)
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
lknn.fit(raw_X_train, l_label_train)

oknn_pred = oknn.predict(raw_X_test)
fknn_pred = fknn.predict(raw_X_test)
bknn_pred = bknn.predict(raw_X_test)
sknn_pred = sknn.predict(raw_X_test)
lknn_pred = lknn.predict(raw_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))


print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')

oknn.fit(smoothed_X_train, o_label_train)
fknn.fit(smoothed_X_train, f_label_train)
bknn.fit(smoothed_X_train, b_label_train)
sknn.fit(smoothed_X_train, s_label_train)
lknn.fit(smoothed_X_train, l_label_train)

oknn_pred = oknn.predict(smoothed_X_test)
fknn_pred = fknn.predict(smoothed_X_test)
bknn_pred = bknn.predict(smoothed_X_test)
sknn_pred = sknn.predict(smoothed_X_test)
lknn_pred = lknn.predict(smoothed_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))



print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')

oknn.fit(translate_X_train, o_label_train)
fknn.fit(translate_X_train, f_label_train)
bknn.fit(translate_X_train, b_label_train)
sknn.fit(translate_X_train, s_label_train)
lknn.fit(translate_X_train, l_label_train)

oknn_pred = oknn.predict(translate_X_test)
fknn_pred = fknn.predict(translate_X_test)
bknn_pred = bknn.predict(translate_X_test)
sknn_pred = sknn.predict(translate_X_test)
lknn_pred = lknn.predict(translate_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))


print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='euclidean')

oknn.fit(final_X_train, o_label_train)
fknn.fit(final_X_train, f_label_train)
bknn.fit(final_X_train, b_label_train)
sknn.fit(final_X_train, s_label_train)
lknn.fit(final_X_train, l_label_train)

oknn_pred = oknn.predict(final_X_test)
fknn_pred = fknn.predict(final_X_test)
bknn_pred = bknn.predict(final_X_test)
sknn_pred = sknn.predict(final_X_test)
lknn_pred = lknn.predict(final_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))

9.67741935483871
35.483870967741936
53.763440860215056
65.59139784946237
43.01075268817204
55.913978494623656
59.13978494623656
67.74193548387096
8.602150537634408
33.333333333333336
40.86021505376344
50.53763440860215
4.301075268817204
29.032258064516128
52.68817204301075
66.66666666666667
18.27956989247312
40.86021505376344
50.53763440860215
60.215053763440864





12.903225806451612
38.70967741935484
55.913978494623656
68.81720430107526
44.086021505376344
55.913978494623656
59.13978494623656
67.74193548387096
9.67741935483871
30.107526881720432
38.70967741935484
50.53763440860215
2.150537634408602
30.107526881720432
52.68817204301075
67.74193548387096
20.43010752688172
41.935483870967744
53.763440860215056
61.29032258064516





0.0
36.55913978494624
49.46236559139785
68.81720430107526
41.935483870967744
53.763440860215056
61.29032258064516
75.26881720430107
15.053763440860216
35.483870967741936
44.086021505376344
52.68817204301075
5.376344086021505
36.55913978494624
54.838709677419